<a href="https://colab.research.google.com/github/MayumyCH/streamlit_data_project_python/blob/main/data_science_project_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto de Ciencia de Datos

Este proyecto va a usar la data generada y analizada del sgte proyecto completo :  

"Predicción de ventas de videojuegos de Vgchartz - plataforma Metacritic" ([Link](https://github.com/MayumyCH/video_game_sales_with_ratings)) 

## 1. ⛓ Importación de librerias

In [ ]:
import pandas as pd # Manejo de data estructurada (Dataframe)
import numpy as np #Manejo de matrices

import matplotlib.pyplot as plt # Gráficas
import seaborn as sns  #Gráficas sencillas

# Separar datos en Train(Entrenamiento) y Test(Prueba)
import sklearn
from sklearn.model_selection import train_test_split 

seed=2020 # Una semilla se usa para que el modelo sea replicable

In [ ]:
print(sklearn.__version__)
!python3 --version

1.0.2
Python 3.7.13


## 2. ⏳ Cargamos nuestro dataset

In [ ]:
url = "https://raw.githubusercontent.com/MayumyCH/streamlit_data_project_python/main/data/data_videojuego.csv"
dataGames = pd.read_csv(url, index_col=False)
dataGames.head(3)

,Unnamed: 0,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,Has_Score,GPlatforms
0,16,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.02,9.09,0.98,3.96,8.0,97.0,50.0,8.2,3994.0,Rockstar North,M,True,Playstation
1,17,Grand Theft Auto: San Andreas,PS2,2004,Action,Take-Two Interactive,9.43,0.40,0.41,10.57,8.0,95.0,80.0,9.0,1588.0,Rockstar North,M,True,Playstation
2,23,Grand Theft Auto V,X360,2013,Action,Take-Two Interactive,9.66,5.14,0.06,1.41,8.0,97.0,58.0,8.1,3711.0,Rockstar North,M,True,Xbox


## 3.🕵️ Features Selection

In [ ]:
# VARIABLE NUMERICA
df_num = dataGames.select_dtypes("number").drop(columns=["Year","Other_Sales","JP_Sales", "EU_Sales"])

# VARIABLE CATEGORICA
df_cat = dataGames[["GPlatforms", "Rating"]]

df_cat = pd.get_dummies(df_cat, drop_first=True)

In [ ]:
dataModel = pd.concat([df_num, df_cat], axis = 1)
X = dataModel.drop(columns="Global_Sales")
y = dataModel["Global_Sales"]

## 4. 📈 Modelado

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=seed)  #Semilla para replicar el modelo

In [ ]:
#Revisamos los tamaños de las pruebas de train y test
print("Tamaño del conjunto de datos Inicial:", dataModel.shape)
print("Tamaño del conjunto de características del entrenamiento:",X_train.shape)
print("Tamaño del conjunto de características de prueba:",X_test.shape)
print("Tamaño de la variable objetivo del entrenamiento:",y_train.shape)
print("Tamaño de la variable objetivo de prueba:",y_test.shape)

Tamaño del conjunto de datos Inicial: (2003, 15)
Tamaño del conjunto de características del entrenamiento: (1402, 14)
Tamaño del conjunto de características de prueba: (601, 14)
Tamaño de la variable objetivo del entrenamiento: (1402,)
Tamaño de la variable objetivo de prueba: (601,)


In [ ]:
# IMPORTANDO LOS DIFERENTES MODELOS DE REGRESION
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

# METRICAS MODELOS
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn import metrics # METRICAS MODELOS

In [ ]:
# EVALUANDO ERRORES
def evaluate_metrics(true, predicted):
    print('-'*50)

    # Calculando el Error
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))

    r2_square = metrics.r2_score(true, predicted)

    # print("Error Absoluto Medio (MAE)") # Que tan alejados estan mis valores reales de los predichos
    print("Linear Regression Performance: MAE = {:.03f}".format(mae))

    print("Linear Regression Performance: MSE = {:.03f}".format(mse)) # No se interpreta

    # print("\nCalculando la Raiz del Error Cuadratico Medio (RMSE)")
    print("Linear Regression Performance: RMSE = {:.03f}".format(rmse)) # Raiz de MSE
    # Que tan alejados estan mis valores reales de los predichos
    # Mas penalizador
    # Diferencia entre los valores reales y los valores predichos

    print("r2_square= {:.02f}".format(r2_square)) 
    # Que tanto estamos logrando predecir el modelo

In [ ]:
def fit_and_evaluate(model, viewMetrics=False):
    
    model.fit(X_train, y_train) # Entrenamiento del modelo
    
    # Predecir y evaluar
    model_pred_train = model.predict(X_train)
    model_pred_test = model.predict(X_test)

    model_rmse= np.sqrt(metrics.mean_squared_error(y_train, model_pred_train)) 
    # Buscar el menor error con el TRAIN

    if viewMetrics:
      print('Calculando los errores en la data del TRAIN')
      evaluate_metrics(y_train,model_pred_train)

      print('\nCalculando los errores en la data del TEST')
      evaluate_metrics(y_test,model_pred_test)
    else:
      print("Rendimiento de Regresión en la deta del TEST: RMSE = {:.02f}".format(model_rmse))


    # Retornar la metrica del rendimiento
    return model_rmse

In [ ]:
# MODELO DUMMY
basic_pred_train = np.median(X_train)
basic_baseline_rmse =  np.sqrt(np.mean((y_train - basic_pred_train)**2))

In [ ]:
# REGRESION LINEAL
lr = LinearRegression() # Creando un objeto de Regresión Lineal 'lm'
lr_rmse = fit_and_evaluate(lr,True)

Calculando los errores en la data del TRAIN
--------------------------------------------------
Linear Regression Performance: MAE = 0.201
Linear Regression Performance: MSE = 0.162
Linear Regression Performance: RMSE = 0.403
r2_square= 0.86

Calculando los errores en la data del TEST
--------------------------------------------------
Linear Regression Performance: MAE = 0.192
Linear Regression Performance: MSE = 0.138
Linear Regression Performance: RMSE = 0.371
r2_square= 0.87


In [ ]:
# SVM - MAQUINA DE SOPORTE VECTORIAL
svm = SVR(C = 1000, gamma=0.1)
svm_rmse = fit_and_evaluate(svm)

Rendimiento de Regresión en la deta del TEST: RMSE = 0.10


In [ ]:
# RANDOM FOREST REGRESOR
random_forest_regresor = RandomForestRegressor(random_state=60)
random_forest_rmse = fit_and_evaluate(random_forest_regresor,True)

Calculando los errores en la data del TRAIN
--------------------------------------------------
Linear Regression Performance: MAE = 0.002
Linear Regression Performance: MSE = 0.000
Linear Regression Performance: RMSE = 0.013
r2_square= 1.00

Calculando los errores en la data del TEST
--------------------------------------------------
Linear Regression Performance: MAE = 0.004
Linear Regression Performance: MSE = 0.000
Linear Regression Performance: RMSE = 0.019
r2_square= 1.00


In [ ]:
# KNN - VECINOS CERCANOS
knn = KNeighborsRegressor(n_neighbors=10)
knn_rmse = fit_and_evaluate(knn,True)

Calculando los errores en la data del TRAIN
--------------------------------------------------
Linear Regression Performance: MAE = 0.046
Linear Regression Performance: MSE = 0.070
Linear Regression Performance: RMSE = 0.265
r2_square= 0.94

Calculando los errores en la data del TEST
--------------------------------------------------
Linear Regression Performance: MAE = 0.049
Linear Regression Performance: MSE = 0.072
Linear Regression Performance: RMSE = 0.268
r2_square= 0.93


## 5. 🔥 Guardando los modelos


In [ ]:
# Exportando nuestro modelo
import dill
dill.settings['recurse'] = True

import pickle # Guardar el flujo

with open("model_rf.pkl", "wb") as f:
    pickle.dump(random_forest_regresor, f) 
with open("model_knn.pkl", "wb") as f:
    pickle.dump(knn, f)
with open("model_svm.pkl", "wb") as f:
    pickle.dump(svm, f)
with open("model_lr.pkl", "wb") as f:
    pickle.dump(lr, f) # Guardar el flujo

print("Se genero todos los pkl de los modelos")

Se genero todos los pkl de los modelos
